In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
# warning : (버전, 업데이트 문제, 에러는 아님) 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
movies = pd.read_csv('/content/drive/MyDrive/YW_AI/양진욱/영화추천/tmdb_5000_movies.csv')

In [4]:
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [5]:
# 필요한 column 만 추출 
movies_df= movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."


In [6]:
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [7]:
type(movies_df['genres'][0])
# str 타입인 것을 확인할 수 있다.

str

### genres, keywords 칼럼들의 str형태를 list형태로 바꿔주기

In [8]:
from ast import literal_eval  # 문자열 파싱 라이브러리

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

# literal_eval 이게 적용된 코드를 적용하면 " -> ' 바뀐 리스트 형태로 나옴

In [9]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [10]:
type(movies_df['genres'][0])
# str타입에서 list타입으로 바뀐 것을 확인할 수 있다.

list

### list 내 여러개 딕셔너리의 name키에 해당하는 값들을 리스트로 변환

In [11]:
movies_df['genres']

0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
1       [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
                              ...                        
4798    [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
4799    [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4801                                                   []
4802                  [{'id': 99, 'name': 'Documentary'}]
Name: genres, Length: 4803, dtype: object

In [12]:
# key 값에 따른 value  값만 추출 
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])

In [13]:
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."


## -> 데이터 전처리 끝

## 장르 컨텐츠 유사도 측정

@장르 CBF 추천 : 장르를 피처 벡터화한 후 행렬 데이터 값을 코사인 유사도로 계산하기
<프로세스>
1. 장르 피처 벡터화: 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환
2. 코사인 유사도 계산 : genres 문자열을 피처 벡터화한 행렬로 변환한 데이터 세트를 
   코사인 유사도로 비교
3. 평점으로 계산 : 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화 추천

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
# 참고 : CountVectorizer에 대하여
# CountVectorizer는 다음 세가지를 수행한다.
# 1. 문서를 토큰 리스트로 변환한다.
# 2. 각 문서에서 토큰의 출현 빈도를 센다.
# 3. 각 문서를 BOW(Bag of Words) 인코딩 벡터로 변환한다.

from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'and': 0,
 'document': 1,
 'first': 2,
 'is': 3,
 'last': 4,
 'one': 5,
 'second': 6,
 'the': 7,
 'third': 8,
 'this': 9}

In [16]:
movies_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [17]:
# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['genres_literal']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres_literal, Length: 4803, dtype: object

In [18]:
# CountVectorizer로 학습시켰더니 4803개 영화에 대한 276개 장르의 '장르 매트릭스'가 생성되었다.
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2)) 
#min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [19]:
count_vect.get_feature_names()

['action',
 'action adventure',
 'action animation',
 'action comedy',
 'action crime',
 'action drama',
 'action family',
 'action fantasy',
 'action history',
 'action horror',
 'action mystery',
 'action romance',
 'action science',
 'action thriller',
 'action war',
 'action western',
 'adventure',
 'adventure action',
 'adventure animation',
 'adventure comedy',
 'adventure crime',
 'adventure documentary',
 'adventure drama',
 'adventure family',
 'adventure fantasy',
 'adventure history',
 'adventure horror',
 'adventure mystery',
 'adventure romance',
 'adventure science',
 'adventure thriller',
 'adventure war',
 'adventure western',
 'animation',
 'animation action',
 'animation adventure',
 'animation comedy',
 'animation drama',
 'animation family',
 'animation fantasy',
 'animation music',
 'animation mystery',
 'animation science',
 'animation war',
 'comedy',
 'comedy action',
 'comedy adventure',
 'comedy animation',
 'comedy crime',
 'comedy documentary',
 'comedy dram

In [20]:
print(genre_mat)

  (0, 0)	1
  (0, 16)	1
  (0, 124)	1
  (0, 232)	1
  (0, 138)	1
  (0, 1)	1
  (0, 24)	1
  (0, 135)	1
  (0, 233)	1
  (1, 0)	1
  (1, 16)	1
  (1, 124)	1
  (1, 24)	1
  (1, 125)	1
  (2, 0)	1
  (2, 16)	1
  (2, 1)	1
  (2, 64)	1
  (2, 20)	1
  (3, 0)	1
  (3, 64)	1
  (3, 90)	1
  (3, 234)	1
  (3, 4)	1
  (3, 68)	1
  :	:
  (4796, 90)	1
  (4796, 234)	1
  (4796, 106)	1
  (4796, 144)	1
  (4797, 234)	1
  (4797, 153)	1
  (4797, 157)	1
  (4798, 0)	1
  (4798, 64)	1
  (4798, 234)	1
  (4798, 4)	1
  (4798, 78)	1
  (4799, 44)	1
  (4799, 214)	1
  (4799, 58)	1
  (4800, 90)	1
  (4800, 44)	1
  (4800, 214)	1
  (4800, 104)	1
  (4800, 50)	1
  (4800, 250)	1
  (4800, 182)	1
  (4800, 251)	1
  (4800, 229)	1
  (4802, 80)	1


In [21]:
# CountVectorizer로 학습시켰더니 4803개 영화에 대한 22개 장르의 '장르 매트릭스'가 생성되었다.
count_vect2 = CountVectorizer(min_df=1, ngram_range=(1,1)) 
#min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 1
genre_mat2 = count_vect2.fit_transform(movies_df['genres_literal'])
print(genre_mat2.shape)

(4803, 22)


In [22]:
genre_mat2.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
count_vect2.get_feature_names()

['action',
 'adventure',
 'animation',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'family',
 'fantasy',
 'fiction',
 'foreign',
 'history',
 'horror',
 'movie',
 'music',
 'mystery',
 'romance',
 'science',
 'thriller',
 'tv',
 'war',
 'western']

In [24]:
print(genre_mat2)

  (0, 0)	1
  (0, 1)	1
  (0, 8)	1
  (0, 17)	1
  (0, 9)	1
  (1, 0)	1
  (1, 1)	1
  (1, 8)	1
  (2, 0)	1
  (2, 1)	1
  (2, 4)	1
  (3, 0)	1
  (3, 4)	1
  (3, 6)	1
  (3, 18)	1
  (4, 0)	1
  (4, 1)	1
  (4, 17)	1
  (4, 9)	1
  (5, 0)	1
  (5, 1)	1
  (5, 8)	1
  (6, 2)	1
  (6, 7)	1
  (7, 0)	1
  :	:
  (4792, 18)	1
  (4792, 12)	1
  (4792, 15)	1
  (4793, 6)	1
  (4794, 18)	1
  (4794, 3)	1
  (4794, 12)	1
  (4795, 6)	1
  (4796, 17)	1
  (4796, 9)	1
  (4796, 6)	1
  (4796, 18)	1
  (4797, 18)	1
  (4797, 10)	1
  (4798, 0)	1
  (4798, 4)	1
  (4798, 18)	1
  (4799, 3)	1
  (4799, 16)	1
  (4800, 6)	1
  (4800, 3)	1
  (4800, 16)	1
  (4800, 19)	1
  (4800, 13)	1
  (4802, 5)	1


In [25]:
# CountVectorizer로 학습시켰더니 4803개 영화에 대한 276개 장르의 '장르 매트릭스'가 생성되었다.
count_vect2 = CountVectorizer(min_df=0, ngram_range=(1,2)) #min_df: 단어장에 들어갈 최소빈도, ngram_range: 1 <= n <= 2
genre_mat2 = count_vect2.fit_transform(movies_df['genres_literal'])
print(genre_mat2.shape)

(4803, 276)


### 코사인 유사도(cosine_similarity)이용해서 영화별 유사도 계산

In [26]:
# 코사인 유사도에 의해 4803개 영화 각각 유사한 영화들이 계산됨
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[:5])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 [0.12598816 0.16903085 0.3380617  ... 0.12598816 0.         0.        ]
 [0.75592895 0.3380617  0.50709255 ... 0.         0.         0.        ]]


In [27]:
# 자료를 정렬하는 것이 아니라 순서만 알고 싶다면 argsort
# 유사도가 높은 영화를 앞에서부터 순서대로 보여줌
# 0번째 영화의 경우 유사도 순서 : 0번, 3494번, 813번, ..., 2401 순서
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # 전체를 -1칸 간격으로
print(genre_sim_sorted_ind[:5])

[[   0 3494  813 ... 3038 3037 2401]
 [ 262    1  129 ... 3069 3067 2401]
 [   2 1740 1542 ... 3000 2999 2401]
 [2195 1850 3316 ...  887 2544 4802]
 [ 102 2995   56 ... 3046 3045 2401]]


In [28]:
genre_sim_sorted_ind

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]])

## 추천 ver1. 장르 코사인 유사도에 의해 영화를 추천하는 함수

### 함수 분석

In [29]:
# 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
title_movie = movies_df[movies_df['title'] == 'Shutter Island']
title_movie

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
439,11324,Shutter Island,"[Drama, Thriller, Mystery]",7.8,6336,81.914696,"[based on novel, island, hurricane, investigat...",World War II soldier-turned-U.S. Marshal Teddy...,Drama Thriller Mystery


In [30]:
# title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
# sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
title_index = title_movie.index.values
title_index

array([439])

In [31]:
similar_indexes = genre_sim_sorted_ind[title_index, :10]
similar_indexes

array([[ 439, 2063, 2773,  462, 4162,  946, 2080,  987, 1036, 1799]])

In [32]:
# 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
# dataframe에서 index로 사용하기 위해서 2차원 array를 1차원 array로 변경
similar_indexes = similar_indexes.reshape(-1)
print(similar_indexes)

[ 439 2063 2773  462 4162  946 2080  987 1036 1799]


In [33]:
movies_df.iloc[similar_indexes]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
439,11324,Shutter Island,"[Drama, Thriller, Mystery]",7.8,6336,81.914696,"[based on novel, island, hurricane, investigat...",World War II soldier-turned-U.S. Marshal Teddy...,Drama Thriller Mystery
2063,1073,Arlington Road,"[Drama, Thriller, Mystery]",7.0,239,12.819447,"[bomb, terrorist, fbi, professor, paranoia, co...",Threats from sinister foreign nationals aren't...,Drama Thriller Mystery
2773,28211,Chloe,"[Drama, Thriller, Mystery]",5.9,498,26.373864,"[toronto, lesbian, remake of french film, susp...",A doctor hires an escort to seduce her husband...,Drama Thriller Mystery
462,14462,The Manchurian Candidate,"[Drama, Thriller, Mystery]",6.2,393,21.394281,"[senator, gulf war, canoe, kuwait, conspiracy,...",When his army unit was ambushed during the fir...,Drama Thriller Mystery
4162,26388,Buried,"[Drama, Thriller, Mystery]",6.6,827,32.841229,"[isolation, coffin, race against time, buried ...",Paul is a U.S. truck driver working in Iraq. A...,Drama Thriller Mystery
946,2649,The Game,"[Drama, Thriller, Mystery]",7.5,1506,62.925175,"[brother brother relationship, birthday, dange...","In honor of his birthday, San Francisco banker...",Drama Thriller Mystery
2080,49517,Tinker Tailor Soldier Spy,"[Drama, Thriller, Mystery]",6.6,840,33.649652,"[spy, cold war, spying, espionage, mole, briti...","Recently-retired MI6 agent, George Smiley is d...",Drama Thriller Mystery
987,69668,Dream House,"[Drama, Thriller, Mystery]",5.8,391,19.267479,"[house fire, extension ladder, last day on job]","Publisher, Will Atenton quits a lucrative job ...",Drama Thriller Mystery
1036,10684,Courage Under Fire,"[Drama, Thriller, Mystery, War]",6.2,232,12.971122,"[gulf war, pilot, u.s. army, praise, rescue mi...","A US Army officer had made a ""friendly fire"" m...",Drama Thriller Mystery War
1799,2057,Original Sin,"[Drama, Thriller, Mystery, Romance]",5.8,198,14.831402,"[women, sex, cuba, eroticism, lover (female), ...",A young man is plunged into a life of subterfu...,Drama Thriller Mystery Romance


### 실제 함수

In [34]:
def find_sim_movie_ver1(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)    
    # 2차원 데이터를 1차원으로 변환
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

### 영화 Gotfather와 장르가 유사한 영화 10개 추천

In [35]:
similar_movies = find_sim_movie_ver1(movies_df, genre_sim_sorted_ind, 'Shutter Island',10)
similar_movies[['title', 'vote_average', 'genres', 'vote_count']]
# 문제 ; 평점 기반으로 추천하고자 하는데, vote_count가 낮은 영화는 제외하고 싶음

[[ 439 2063 2773  462 4162  946 2080  987 1036 1799]]


,title,vote_average,genres,vote_count
439,Shutter Island,7.8,"[Drama, Thriller, Mystery]",6336
2063,Arlington Road,7.0,"[Drama, Thriller, Mystery]",239
2773,Chloe,5.9,"[Drama, Thriller, Mystery]",498
462,The Manchurian Candidate,6.2,"[Drama, Thriller, Mystery]",393
4162,Buried,6.6,"[Drama, Thriller, Mystery]",827
946,The Game,7.5,"[Drama, Thriller, Mystery]",1506
2080,Tinker Tailor Soldier Spy,6.6,"[Drama, Thriller, Mystery]",840
987,Dream House,5.8,"[Drama, Thriller, Mystery]",391
1036,Courage Under Fire,6.2,"[Drama, Thriller, Mystery, War]",232
1799,Original Sin,5.8,"[Drama, Thriller, Mystery, Romance]",198


In [36]:
# 평점 기준으로 정렬(sort_values)해보니 평가횟수(vote_count)가 낮은 것들도 포함됨. 
# 평가횟수 반영한 평점(가중평점)이 필요함, 내림차순으로 상위 10개 추출 
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


## 가중평점(평점&평가횟수) 반영한 영화 추천

@ 가중평점(Weighted Rating):

   (v/(v+m))*R + (m/(v+m))*C

- v : 영화별 평점을 투표한 횟수(vote_count) ★ 투표횟수가 많은 영화에 가중치 부여
- m : 평점을 부여하기 위한 최소 투표 횟수 -> 여기서는 투표수 상위 60%
- R : 개별 영화에 대한 평균 평점(vote_average)
- C : 전체 영화에 대한 평균 평점(movies_df['vote_average'].mean()
# C, m은 고정값
# v,R은 영화마다 변동값

In [37]:
# 상위 60%에 해당하는 vote_count를 최소 투표 횟수인 m으로 지정
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

In [38]:
# C: 전체 영화에 대한 평균평점 = 약 6점
# m: 평점을 부여하기 위한 최소 투표 횟수 = 370회(상위 60% 수준)
print('C:',round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


### 가중평점을 계산하는 함수

In [39]:
def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

In [40]:
# 기존 데이터에 가중평점 칼럼 추가
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [41]:
# 가중평점(weighted_vote) 기준으로 영화 순위 뽑아보기(내림차순)
movies_df[['weighted_vote','title','vote_average','vote_count','genres']].sort_values('weighted_vote', ascending=False)[:10]

,weighted_vote,title,vote_average,vote_count,genres
1881,8.396052,The Shawshank Redemption,8.5,8205,"[Drama, Crime]"
3337,8.263591,The Godfather,8.4,5893,"[Drama, Crime]"
662,8.216455,Fight Club,8.3,9413,[Drama]
3232,8.207102,Pulp Fiction,8.3,8428,"[Thriller, Crime]"
65,8.136930,The Dark Knight,8.2,12002,"[Drama, Action, Crime, Thriller]"
1818,8.126069,Schindler's List,8.3,4329,"[Drama, History, War]"
3865,8.123248,Whiplash,8.3,4254,[Drama]
809,8.105954,Forrest Gump,8.2,7927,"[Comedy, Drama, Romance]"
2294,8.105867,Spirited Away,8.3,3840,"[Fantasy, Adventure, Animation, Family]"
2731,8.079586,The Godfather: Part II,8.3,3338,"[Drama, Crime]"


## 추천 ver2. 먼저 장르 유사성 높은 영화 20개 선정 후, 가중평점순 10개 선정

In [42]:
def find_sim_movie_ver2(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 index는 제외(기존에 본 영화는 제외하고 추천, title index가 겹치지 않는 영화 추천)
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

### 영화 Gotfather에 대해 장르 유사성, 가중평점 반영한 추천 영화 10개를 뽑아보자

In [43]:
similar_movies = find_sim_movie_ver2(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote', 'genres', 'vote_count']]

,title,vote_average,weighted_vote,genres,vote_count
2731,The Godfather: Part II,8.3,8.079586,"[Drama, Crime]",3338
1847,GoodFellas,8.2,7.976937,"[Drama, Crime]",3128
3866,City of God,8.1,7.759693,"[Drama, Crime]",1814
1663,Once Upon a Time in America,8.2,7.657811,"[Drama, Crime]",1069
883,Catch Me If You Can,7.7,7.557097,"[Drama, Crime]",3795
281,American Gangster,7.4,7.141396,"[Drama, Crime]",1502
4041,This Is England,7.4,6.739664,"[Drama, Crime]",363
1149,American Hustle,6.8,6.717525,"[Drama, Crime]",2807
1243,Mean Streets,7.2,6.626569,"[Drama, Crime]",345
2839,Rounders,6.9,6.530427,"[Drama, Crime]",439


## 요약 : Gotfather를 좋아하는 사람에게 영화 추천해주기

Gotfather장르가 Drama, Crime이다. 
우선 Drama, Crime 장르 기준으로 상위 20개 영화를 뽑아보고, 
그 중 평가횟수를 반영한 가중평점 기준 상위 10개 영화를 뽑아서 추천해준다.

### 응용 : Spider-Man 3 좋아하는 사람 기준으로 장르가 유사한 영화를 추천해주자

In [44]:
similar_movies = find_sim_movie_ver2(movies_df, genre_sim_sorted_ind, 'Spider-Man 3',10)
similar_movies[['title', 'vote_average', 'weighted_vote', 'genres', 'vote_count']]

,title,vote_average,weighted_vote,genres,vote_count
329,The Lord of the Rings: The Return of the King,8.1,8.011871,"[Adventure, Fantasy, Action]",8064
262,The Lord of the Rings: The Fellowship of the Ring,8.0,7.922175,"[Adventure, Fantasy, Action]",8705
330,The Lord of the Rings: The Two Towers,8.0,7.910111,"[Adventure, Fantasy, Action]",7487
19,The Hobbit: The Battle of the Five Armies,7.1,7.027274,"[Action, Adventure, Fantasy]",4760
98,The Hobbit: An Unexpected Journey,7.0,6.961224,"[Adventure, Fantasy, Action]",8297
126,Thor: The Dark World,6.8,6.748873,"[Action, Adventure, Fantasy]",4755
30,Spider-Man 2,6.7,6.652034,"[Action, Adventure, Fantasy]",4321
129,Thor,6.6,6.572735,"[Adventure, Fantasy, Action]",6525
20,The Amazing Spider-Man,6.5,6.478296,"[Action, Adventure, Fantasy]",6586
38,The Amazing Spider-Man 2,6.5,6.466812,"[Action, Adventure, Fantasy]",4179


### 응용 : Enemy at the Gates 좋아하는 사람 기준으로 장르가 유사한 영화를 추천해주자

In [45]:
similar_movies = find_sim_movie_ver2(movies_df, genre_sim_sorted_ind, 'Enemy at the Gates',10)
similar_movies[['title', 'vote_average', 'weighted_vote', 'genres', 'vote_count']]

,title,vote_average,weighted_vote,genres,vote_count
1525,Apocalypse Now,8.0,7.708775,"[Drama, War]",2055
2798,The Boy in the Striped Pyjamas,7.7,7.373173,"[War, Drama]",1451
2536,The Deer Hunter,7.8,7.310348,"[Drama, War]",921
1662,Glory,7.4,6.757199,[War],383
585,War Horse,7.0,6.753283,"[Drama, War]",992
2662,Sarah's Key,7.2,6.475666,"[Drama, War]",196
2016,The Water Diviner,6.8,6.472943,"[War, Drama]",431
557,Jarhead,6.6,6.434392,"[Drama, War]",765
2671,Born on the Fourth of July,6.7,6.405936,"[Drama, War]",395
3310,Far from Men,6.6,6.145019,"[Drama, War]",43
